# Project 3 - Data Delivery, Backend, Visualization
## The Data extraction and Transformation Module

In [2]:
# Dependencies
import pandas as pd
import requests
import json
from pymongo import MongoClient
from us import states


# Census API Key
from census import Census
from config import api_key
c = Census(api_key, year=2021)

## Get Population, Income and Per-Capita Data via API

In [4]:

years=[2018,2019,2020]
complete_years=pd.DataFrame()

for curr_year in years:
    census_data = c.acs5.state_county(fields = ('NAME', "B01003_001E", "B01002_001E", "B19301_001E", "B17001_002E"),
                                      state_fips = "*",
                                      county_fips = "*", year = curr_year)

    census_data_curr_year=pd.DataFrame(census_data)
    # Column Reordering
# https://api.census.gov/data/2019/acs/acs5/variables.html
    census_data_curr_year.rename(columns={"B01003_001E": "Population",
                                     "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                     "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name"},inplace=True)

    census_data_curr_year["Year"]=curr_year
    frames=[complete_years,census_data_curr_year]
    complete_years=pd.concat(frames)
complete_years.drop()
complete_years.head()

,Name,Population,Median Age,Per Capita Income,Poverty Count,state,county,Year
0,"Mineral County, Montana",4211.0,52.6,23744.0,705.0,30,061,2018
1,"Petroleum County, Montana",432.0,52.1,32565.0,35.0,30,069,2018
2,"Powell County, Montana",6861.0,45.3,25060.0,666.0,30,077,2018
3,"Sanders County, Montana",11521.0,53.2,23822.0,2210.0,30,089,2018
4,"Wibaux County, Montana",1175.0,46.0,23137.0,180.0,30,109,2018


In [11]:
json_years_str = complete_years.to_json(orient="records")
# json_years = json.loads(json_years_str)
with open("../Data/popincome_data.json", "w") as outfile:
    outfile.write(json_years_str)

## Exract Insurance Data

In [13]:
api_string=f"&key={api_key}"
base_url="https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_PT,NUI_PT,NAME&for=county:*&in=state:*&"
insurance_data=[]
complete_insurance_df=pd.DataFrame()
for curr_year in years:
    time=f"time={curr_year}"
    url=base_url+time+api_string
    curr_data = requests.get(url).json()
    insurance_data.append(curr_data)
    names=curr_data[0]
    curr_insurance_df=pd.DataFrame(curr_data,columns=names)
    curr_insurance_df=curr_insurance_df.drop(curr_insurance_df.index[0])
    frames=[complete_insurance_df,curr_insurance_df]
    complete_insurance_df=pd.concat(frames)

In [15]:
complete_insurance_str = complete_insurance_df.to_json(orient="records")
complete_insurance_json = json.loads(complete_insurance_str)
with open("../Data/insurance_data.json", "w") as outfile:
    outfile.write(complete_insurance_str)

## Loading Json Data into MongoDB

In [ ]:
# Making Connection
myclient = MongoClient("mongodb://localhost:27017/")
  
# database
db = myclient["CensusData"]
  
# Created or Switched to collection
# names: GeeksForGeeks
Collection = db["counties_geocode"]
 
# # Loading or Opening the json file
with open('Data/gz_2010_us_050_00_5m_rows.json') as file:
    file_data = json.load(file)
     
# Inserting the loaded data in the Collection
# if JSON contains data more than one entry
# insert_many is used else insert_one is used
if isinstance(file_data, list):
    Collection.insert_many(file_data) 
else:
    Collection.insert_one(file_data)